# Overview

This tutorial will show you how to:
  1. Train BPNet
  2. Compute contribution scores
  3. Discover motifs with TF-MoDISco
  4. Determine motif instances with CWM scanning
  5. Simulate motif spacing
  

This will be done on a subset of the data from the [BPNet paper](https://www.biorxiv.org/content/biorxiv/early/2019/08/21/737981.full.pdf) measuring TF binding of 3/4 TFs (Oct4, Sox2, and Nanog) with ChIP-nexus in mouse embryonic stem cells (mESCs). To make things faster, we will only use peak regions from chromosomes 2, 16, 17, 18 and 19 to train/evaluate the model and run TF-MoDISco (**10% of the original data**).

We'll be using the `bpnet` python package to accomplish these steps. You can find out more about it at https://github.com/kundajelab/bpnet.

Use the 'Table of contents' on the left to navigate this notebook. If you have any suggestions or questions, you can add comments to the individual cells using the Ctrl+Alt+M shortcut.




## Setup

Make sure you have enabled the GPU runtime by navigating to the menu 'Runtime', select 'Change runtime type' and set the runtime to 'GPU'.

### Install dependencies

In [1]:
# !pip uninstall tensorflow -y

# !pip install tensorflow==1.14.0


# NOTE: after running this, restart the runtime: Runtime -> Restart runtime -> yes

In [2]:
# !pip install pprint -i https://pypi.doubanio.com/simple/ --trusted-host pypi.douban.com
# !pip install bpnet --quiet --quiet

In [3]:
# !apt-get install -y bedtools > /dev/null
# !pip install git+https://github.com/kundajelab/DeepExplain.git --quiet
# !pip install -U cloudpickle h5py tqdm --quiet
# !pip install -U pyyaml --quiet
# !pip install bpnet --quiet --quiet
# !pip install -U jupyter_client>=6.1.2 --quiet
# !pip install wandb snakemake --quiet

# %env HDF5_USE_FILE_LOCKING=FALSE

If you are running this on your own machine, please see the installation intructions at https://github.com/kundajelab/bpnet.

In [4]:
# %tensorflow_version 1.x
import bpnet
from bpnet.cli.contrib import ContribFile
from bpnet.plot.tracks import plot_tracks, to_neg

import uuid
import datetime
import seaborn as sns
import matplotlib.pyplot as plt
from IPython.display import HTML
import pandas as pd
import numpy as np

Using TensorFlow backend.
/home/ubuntu/miniconda3/envs/bpnet/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/ubuntu/miniconda3/envs/bpnet/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/ubuntu/miniconda3/envs/bpnet/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/ubuntu/minicon

2021-06-13 21:04:23,720 [WARNING] From /home/ubuntu/miniconda3/envs/bpnet/lib/python3.6/site-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



#### Optional: Setup wandb

In [5]:
# Uncomment if you would like to run this notebook using Wandb

# import wandb
# wandb.init(project='bpnet-demo', entity='avsec')

### Get test data

In [6]:
# # Dowload and extract the bpnet repo
# !wget 'https://drive.google.com/uc?authuser=0&id=1YX1svzlRnLtvxyzj5O4B1l5-_wArZ9Q6&export=download' -O bpnet.tar.gz
# !tar xvfz bpnet.tar.gz  > /dev/null

In [7]:
# # Download chip_nexus data
# !snakemake -d bpnet/examples -s bpnet/examples/Snakefile chip_nexus -j 4 --quiet

# Train BPNet

<img src="https://github.com/kundajelab/bpnet/blob/master/notebooks/figs/bpnet-arch.png?raw=1" alt="BPNet" style="width: 450px;"/>

## 1. Specify data -> write `dataspec.yml`

BPNet takes as input nucleotide sequence and outputs the read coverage profile for multiple tracks at base-resolution. The coverage tracks can come from any genome-wide functional genomics assay that has a sufficient spatial resolution including ChIP-nexus, ChIP-exo, ChIP-seq, DNase-seq, and ATAC-seq. Additionally, different experiments may have differnet biases that need to be accounted for using the control or bias experiments. Both, the signal and the bias/control tracks have to be stored in [BigWig](https://genome.ucsc.edu/goldenpath/help/bigWig.html) files.
 

The file paths to the BigWig tracks on which to train the model should be specified in a `dataspec.yml` as follows:

In [8]:
from pathlib import Path
exp_dir = Path('bpnet/examples/chip-nexus/') 

!cat {exp_dir}/dataspec.yml

fasta_file: ../data/mm10.subset.fa  # reference genome fasta file
task_specs:  # specifies multiple tasks (e.g. Oct4, Sox2 Nanog)

  Oct4:
    tracks:
      - ../data/chip-nexus/Oct4/counts.pos.subset.bw
      - ../data/chip-nexus/Oct4/counts.neg.subset.bw
    peaks: ../data/chip-nexus/Oct4/idr-optimal-set.summit.subset.bed.gz
  Sox2:
    tracks:
      - ../data/chip-nexus/Sox2/counts.pos.subset.bw
      - ../data/chip-nexus/Sox2/counts.neg.subset.bw
    peaks: ../data/chip-nexus/Sox2/idr-optimal-set.summit.subset.bed.gz

  Nanog: # Nanog is the task name
    tracks:
      # List of bigwig files (1 or more) corresponding to the task
      # The model will predict each track individually (here coverage of
      # reads mapping to the positive and negative strand) and
      # the contribution scores will be averaged across all of these tracks
      - ../data/chip-nexus/Nanog/counts.pos.subset.bw
      - ../data/chip-nexus/Nanog/counts.neg.subset.bw

    # Peaks associated with Nanog task

### Data stats

In [9]:
# chromsomome names of differnet peaks
!zcat bpnet/examples/data/chip-nexus/*/idr-optimal-set.summit.subset.bed.gz | cut -f 1 | sort -u

chr16
chr17
chr18
chr19
chr2


Each task (or TF) can specify a set of peaks associated with it. Here are the number of peaks per TF we will use in this tutorial:

In [10]:
tasks = ['Oct4', 'Sox2', 'Nanog']

# number of peaks per task
for task in tasks:
  print(task)
  !zcat bpnet/examples/data/chip-nexus/{task}/idr-optimal-set.summit.subset.bed.gz | wc -l

Oct4
5373
Sox2
2265
Nanog
12007


### FAQ

#### How can I visualize the raw data before training the model?

Glad you asked. Before you jump ahead and start training the model, we recommend eyeballing the coverage tracks (BigWig) and peak regions (bed) using the genome browser such as the [WashU](https://epigenomegateway.wustl.edu/) or [IGV](https://software.broadinstitute.org/software/igv/). If you can not identify peaks by eye then the model will not be able to do it either.

Having specified your data in `dataspec.yml`, you can use also `bpnet.specs.DataSpec` to parse the file and visualize the tracks for a specific genomic interval in your jupyter notebook.

#### How do I get my data into a BigWig file?

Functional genomics experiments based on sequencing yield many short reads which then get aligned to the reference genome. The alignment locations of the reads are typically stored in the [BAM](http://samtools.github.io/hts-specs/SAMv1.pdf) file. There are different ways of computing the coverage tracks from aligned reads. To prevent loosing any spatial information in the profiles, we would like to generate non-smoothed tracks (as raw as possible). For ChIP-exo/nexus/seq experiments this means counting the 5' locations of the reads. Note that the aligned reads also have strand information hence you should generate two coverage tracks, one for the positive/forward and one for the negative/reverse strand. If multiple technical or biological replicate experiments were performed for a specific transcription factor, we recommend to add up their coverage (for example by merging the BAM files or adding the coverage tracks of the BigWig files).

Here are the commands to do the conversion from BAM to BigWig for both strands:

1. Sort the bam file: `samtools sort alignments.pos.bam alignments.sorted.bam`
2. Convert BAM to bedGraph for positive and strand:
```
bedtools genomecov -5 -bg -strand + -ibam alignments.sorted.bam | sort -k1,1 -k2,2n > alignments.pos.bedGraph
bedtools genomecov -5 -bg -strand - -ibam alignments.sorted.bam | sort -k1,1 -k2,2n > alignments.neg.bedGraph
```
3. Convert begGraph to bigwig using bedGraphToBigWig from UCSC ([download link](http://hgdownload.soe.ucsc.edu/admin/exe/linux.x86_64/bedGraphToBigWig)):
```
bedGraphToBigWig alignments.pos.bedGraph genome.chromosome-sizes.txt alignments.pos.bw
bedGraphToBigWig alignments.neg.bedGraph genome.chromosome-sizes.txt alignments.neg.bw
```

`genome.chromosome-sizes.txt` is the genome file containing `chromosome_name<TAB>chromosome_length` entries.

See also the [Snakemake rules](https://github.com/kundajelab/bpnet-manuscript/blob/d7af1bda3ac8cc342b32f9cdac481ba55fe7ddca/src/bpnet-pipeline/prepare-data.smk#L99-L133) for conversion from TagAlign files instead of BAM files and the [ChIP-nexus pipeline](https://github.com/kundajelab/chip-nexus-pipeline/blob/ea40683d65b9317843a7fbdcc75bd33e481bee8e/src/encode_count_signal_track.py#L29-L60).


#### How do I get `regions.bed`?

For large genomes such as human or mouse, training genome-wide can be computationally expensive. Most of the regions in the genome will contain very little counts, hence the model will not recieve a lot of information. We can significantly speed up the training process by training the model only in regions with higher number of counts. These regions are determined using traditional peak callers such as MACS2. Since we just want to discard regions with little or no counts, we don't care about the exact peak locations or even high false positive rates. Hence almost any peak caller should be fine.

#### Can I train the model without the bias track?

Technically, yes. It will work well for assays with low amount of bias such as ChIP-exo or ChIP-nexus. However, we generally recommend using the bias track. 

#### Can I train the model with differnet assays simultaneusly?

Yes. If you are using different assays with similar resolution (e.g. ChIP-nexus and ChIP-exo), you can just specify the bigwig files and use different bias/control files for differnt subsets of the tracks. If you are using different assays with different resolutions, you might want to tweak the parameters of model output heads to best fit the individual experiments.

#### Should I train a single multi-task model or multiple single-task models?

If you expect the tracks to share some sequence motifs, it's likely beneficial to train a multi-task model (e.g. use a single `dataspec.yml`). Also, handling a single model is more convenient than handling multiple models. However, if you have many tracks it might be challenging to train a multi-task model.

## 2. Train the model

Having specified `dataspec.yml`, we are now ready to train the model with 

```
bpnet train <dataspec.yml> <output dir> [optional flags]`
```


We will use a pre-made model [bpnet9](../bpnet/premade/bpnet9.gin) as a starting point and modify a few parameters specified in the config.gin file. Specifically, we will 
- train the model only on chromosomes 16-19
- evaluate the model on chromosome 2
- use only 3 layers of dilated convolutions 
- use an input sequence length of 200 bp and accordingly lower the augmentation shift to 100 bp

In [11]:
!cat {exp_dir}/config.gin
# NOTE: test_chr will be also excluded similar to 'exclude_chr'

# exclude a large portion of the training set
exclude_chr=["chrX","chrY","chr5","chr6","chr7","chr10","chr14","chr11","chr13","chr12","chr15"]
valid_chr = ['chr2']
test_chr = ['chr1', 'chr8', 'chr9',
            'chr3', 'chr4']
seq_width = 200
n_dil_layers = 3
bpnet_data.interval_augmentation_shift = 100
train.seed = 1

Have a look at the original gin file of bpnet9 here: https://github.com/kundajelab/bpnet/blob/master/bpnet/premade/bpnet9-ginspec.gin. For more information on using gin files see <https://github.com/google/gin-config>. 

To track model training and evaluation, we will use [wandb](http://wandb.com/) by adding `--wandb=avsec/bpnet-demo` to `bpnet train`. You can navigate to https://app.wandb.ai/avsec/bpnet-demo to see the training progress.

Let's train!

In [12]:
# setup all the file paths
model_dir = exp_dir / 'output'
contrib_file = model_dir/'contrib.deeplift.h5'
contrib_null_file = model_dir/'contrib.deeplift.null.h5'
modisco_dir = model_dir/'modisco'

In [13]:
# setup a new run_id (could be done automatically, but then the output directory would change)
run_id = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S") + "_" + str(uuid.uuid4())

# Train for at most 10 epochs
!cd {exp_dir} && bpnet train dataspec.yml --premade=bpnet9 --config=config.gin . --override='train.epochs=10' --run-id '{run_id}' --wandb=avsec/bpnet-demo --in-memory

# softlink the new output directory
!rm {exp_dir}/output && ln -srf {exp_dir}/{run_id}  {exp_dir}/output

Using TensorFlow backend.
/home/ubuntu/miniconda3/envs/bpnet/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/ubuntu/miniconda3/envs/bpnet/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/ubuntu/miniconda3/envs/bpnet/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/ubuntu/minicon

INFO [06-13 21:04:42] Loading the training data into memory
100%|███████████████████████████████████████████| 99/99 [00:03<00:00, 27.78it/s]
INFO [06-13 21:04:46] Loading the validation data into memory
100%|███████████████████████████████████████████| 99/99 [00:04<00:00, 24.46it/s]
2021-06-13 21:04:54.941854: W tensorflow/compiler/jit/mark_for_compilation_pass.cc:1412] (One-time warning): Not using XLA:CPU for cluster because envvar TF_XLA_FLAGS=--tf_xla_cpu_global_jit was not set.  If you want XLA:CPU, either set that envvar, or use experimental_jit_scope to enable XLA:CPU.  To confirm that XLA is active, pass --vmodule=xla_compilation_cache=1 (as a proper command-line flag, not via TF_XLA_FLAGS) or set the envvar XLA_FLAGS=--xla_hlo_profile.
Epoch 1/10
98/98 [==============================] - 14s 141ms/step - loss: 1071.9160 - Oct4/profile_loss: 275.0354 - Oct4/counts_loss: 3.2327 - Sox2/profile_loss: 143.4014 - Sox2/counts_loss: 1.2697 - Nanog/profile_loss: 590.6003 - Nanog/counts_

Executing:  10%|███▏                           | 3/29 [00:04<00:37,  1.43s/cell]2021-06-13 21:07:20.305316: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 AVX512F FMA
2021-06-13 21:07:20.326385: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2700000000 Hz
2021-06-13 21:07:20.328321: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x55a284665a40 executing computations on platform Host. Devices:
2021-06-13 21:07:20.328369: I tensorflow/compiler/xla/service/service.cc:175]   StreamExecutor device (0): <undefined>, <undefined>
Executing: 100%|██████████████████████████████| 29/29 [00:32<00:00,  1.12s/cell]
[NbConvertApp] WARNING | Config option `kernel_spec_manager_class` not recognized by `NbConvertApp`.
[NbConvertApp] Converting notebook evaluate.ipynb to html
[NbConvertApp] Writing 2267132 bytes to evaluate.html
INFO [06-13 21:07:50] Done training and ev

In [41]:
# To see `bpnet train` docs run the following cell
!bpnet train -h

Using TensorFlow backend.
/home/ubuntu/miniconda3/envs/bpnet/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/ubuntu/miniconda3/envs/bpnet/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/ubuntu/miniconda3/envs/bpnet/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/ubuntu/minicon

### View the evaluation results

`bpnet train` produces the following output files:

In [42]:
!ls -latr {model_dir}/

ls: cannot access 'bpnet/examples/chip-nexus/output/': No such file or directory


The main output files are:

- model.h5 - Keras model HDF5 file
- seq_model.pkl - Serialized SeqModel. This is the main trained model.
- eval-report.ipynb/.html - evaluation report containing training loss curves and some example model predictions.
- model.gin -> copied from the input
- dataspec.yaml -> copied from the input

看 evaluation 直接点这[http://132.232.36.144:9090/notebooks/bpnet/examples/chip-nexus/output/evaluate.ipynb]

In [43]:
HTML(filename=model_dir / 'evaluate.html')

FileNotFoundError: [Errno 2] No such file or directory: 'bpnet/examples/chip-nexus/output/evaluate.html'

## 3. Tweak the model

### Modify the hyper-parameters

The first step you can do to improve your model is to adapt the hyper-parameters of the existing model. Have a look at the default hyper-parameters of the pre-made model you were using here: [../bpnet/premade/bpnet9.gin](../bpnet/premade/bpnet9.gin). You can directly override the hyper-parameters from the command line as follows:

```bash
bpnet train dataspec.yaml --premade=bpnet9 --override='train.lr=0.05;model.multi_task_model.n_layers=5' -o trained_model/
```

This will use a different learning rate (0.05) and less convolutional layers (5) by overriding the original values. Note that multiple parameter specifications were separated using `;`. If you find it impractical to specify the hyper-parameters from the CLI, you can instead specify them in the `model.gin` config file: 

```python
train.lr = 0.05
model.multi_task_model.n_layers = 5
```

and then run `bpnet train` as follows:

```bash
bpnet train dataspec.yaml --premade=bpnet9 --config=model.gin -o trained_model/
```

Note that you can use `--override` and `--config` simultaneusly. If both specify the same parameter, then the one specified by `--override` will be used. For example, the following command will use the learning rate of 0.01.

```bash
bpnet train dataspec.yaml --premade=bpnet9 --config=model.gin --override='train.lr=0.01' -o trained_model/
```

Altogether, `--config` overrides the parameters specified by `--premade`; `--override` overrides parameters specified by `--config` and `--premade` (e.g. `--override` > `--config` > `--premade`).

### Specify your own model architecture, loss function or training procedure

Have a look at the [gin-config documentation](https://github.com/google/gin-config) to learn more about the gin config files. This will help you understand how you can utilize them effectively and thereby go beyond the premade models. To specify your own architecture, use a different loss function or a different training procedure, you have to do three things:
1. Implement a python function returning the `bpnet.seqmodel.SeqModel` object.
2. Decorate the function with `@gin.configurable`
3. Specify that you would like to use this model in the `model.gin` config file.

#### How do I specify my own jupyter notebook for evaluation?

You can specify your own jupyter notebook by specifying the `train.eval_report='<path>.ipynb'` in either `config.gin` file or by specifying `--override='train.eval_report="<path>.ipynb"'`. Make sure to copy the first cell of https://github.com/kundajelab/bpnet/blob/master/bpnet/templates/evaluate.ipynb to your template notebook as it contains the following cell metadata: `{"tags": ["parameters"]}`. See https://github.com/nteract/papermill#parameterizing-a-notebook on how to set this yourself.

#### What is `bpnet.seqmodel.SeqModel`?

`SeqModel` is a wrapper around the Keras model. It requires the input to be a one-hot-encoded DNA sequence and it consists of two key components: body and heads. Body will process the input sequence with multiple layers yielding the bottleneck activation map which can be seens as containing an embedding at each nucleotide position. Heads will take the bottleneck activation map as input and they will output the final model predictions. Heads specify the loss function and the evaluation metric. They can additionally accept the bias/control track as input and control for it in the loss function. The benefit of restricting the model architecture in that way is that the sequence contribution scores can be automatically computed with no extra code. Have a look at the `SeqModel` [source code](../bpnet/seqmodel.py) to learn more about it. 

# Compute contribution scores

In [ ]:
# contribution scores
!bpnet contrib {model_dir} --method=deeplift --memfrac-gpu=1 --contrib-wildcard='*/profile/wn' {contrib_file}

In [ ]:
# null contribution scores obtained by shuffling the sequences.
!bpnet contrib {model_dir} --method=deeplift --memfrac-gpu=1 --shuffle-seq --max-regions 5000 --contrib-wildcard='*/profile/wn' {contrib_null_file}

In [ ]:
# To see `bpnet contrib` docs run the following cell
!bpnet contrib -h

### Visualize the contribution scores

Previous command generates an HDF5 file `contrib_file` containing the contribution scores. You can access the the values stored in this file by using the `ContribFile` class.

In [ ]:
import seaborn as sns
from bpnet.cli.contrib import ContribFile
from bpnet.plot.tracks import plot_tracks, to_neg
import seaborn as sns
import matplotlib.pyplot as plt

cf = ContribFile(contrib_file)

In [ ]:
# get chip-nexus profiles and contribution scores from the ContribFile
profiles = cf.get_profiles()
contrib_scores = cf.get_contrib()

In [ ]:
# get example idx with most chip-nexus counts for each task
examples = list({v.max(axis=-2).mean(axis=-1).argmax() for k,v in profiles.items()})
examples

In [ ]:
tasks = ['Oct4', 'Sox2', 'Nanog']

In [ ]:
xrange = slice(50, 150)
for idx in examples:
  plot_tracks({**{'profile/' + k: to_neg(v[idx,xrange]) for k,v in profiles.items()},
               **{'contrib/' + k:v[idx,xrange] for k,v in contrib_scores.items()}},
             title=idx,
             rotate_y=0,
             fig_width=10,
             fig_height_per_track=1);
  sns.despine(top=True, right=True, bottom=True)

Have a look at the [ContribFile API](https://github.com/kundajelab/bpnet/blob/0cb7277b736260f8b4084c9b0c5bd62b9edb5266/bpnet/cli/contrib.py#L262) to explore all the available methods.

 ## Export BigWig files containing contribution scores and model predictions

In [ ]:
!bpnet export-bw {model_dir} {model_dir}/bigwigs/ --contrib-method=deeplift --scale-contribution
# scale-contribution will multiply the contribution scores with total count predictions
# this will ensure that regions without high counts won't get high contribution scores
# We generally recommend using --scale-contribution

In [ ]:
!ls -latr {model_dir}/bigwigs/

You  could visualize these files in the genome browser.

# Run TF-MoDISco to discover motifs

Next step is to cluster the contribution scores into motifs using TF-MoDISco. At first, we will cluster the Oct4 contribution scores in Oct4 peaks (`--only-task-regions`).

In [ ]:
task = 'Oct4'

In [ ]:
# Run modisco only for the Oct4 task
!bpnet modisco-run {contrib_file} --null-contrib-file={contrib_null_file} --contrib-wildcard={task}/profile/wn --premade=modisco-50k --override='TfModiscoWorkflow.min_metacluster_size=1000' --only-task-regions {modisco_dir}/{task}/ --overwrite

In [ ]:
# View the modisco results
HTML(filename=modisco_dir / 'Oct4/modisco.html')

## Visualizing modisco results

Here is an example how you can access the `modisco.h5` file in python and visualize the results yourself.

In [ ]:
# Modisco generated the following files for each TF
!ls -latrh {modisco_dir}/{task}

In [ ]:
from bpnet.modisco.files import ModiscoFile, ModiscoFileGroup

In [ ]:
# Alternative way of loading the files
mf = ModiscoFile(modisco_dir /  'Oct4/modisco.h5')

for p in mf.patterns():
  n_seqlets = mf.n_seqlets(p.name)
  p.plot("seq_ic", title=f"{p.name} ({n_seqlets})")
  plt.ylim([0, 2])
  sns.despine(top=True, bottom=True, right=True)

## Visualize the ChIP-nexus heatmaps 

In [ ]:
# load seqlets for the main motif
mf = ModiscoFile(modisco_dir /  'Oct4/modisco.h5')
seqlets = mf._get_seqlets('metacluster_0/pattern_0')

# load the ContribFile
# NOTE: we have to load it using `from_modisco_dir` since
# modisco was executed only on a subset of the
# regions present in ContribFile
cf = ContribFile.from_modisco_dir(modisco_dir / 'Oct4')

# extract into `StackedSeqletContrib`
sc = cf.extract(seqlets)

In [ ]:
sc.plot(kind='profile_agg', figsize_tmpl=(3.3, 2));
sc.plot(kind='profile', figsize=(10, 8));
sc.plot(kind='seq',figsize_tmpl=(13, 10));
plt.title("Sequence");
# sc.plot(kind='contrib', figsize=(10, 8));

## Generating further reports

For analyzing ChIP-nexus/exo/seq data, there exist an additional report generated by `bpnet chip-nexus analysis` after running modisco.

In [ ]:
# generate more extensive reports specific for ChIP-nexus/exo data
# We'll use a slightly smaller footprint width since the seqlets may run out of chromosomes instead
!bpnet chip-nexus-analysis {modisco_dir}/Oct4 --footprint-width=150

In [ ]:
# This prouced new files to the output directory
!ls -latr {modisco_dir}/Oct4/

In [ ]:
# NOTE: the heatmap files will not be displayed here since 
# the file paths of the plots are not correct.
HTML(f"{modisco_dir}/Oct4/modisco-chip.html")

## Running modisco for the remaining tasks

Next, we will run modisco on the contribution scores for the remaining tasks: Sox2 and Nanog.

In [ ]:
tasks = ['Oct4', 'Sox2', 'Nanog']

In [ ]:
# Run modisco only for the Nanog task
for task in tasks:
  if task == 'Oct4':
    # already exists from before
    continue
  print(f"task: {task}")
  !bpnet modisco-run {contrib_file} --null-contrib-file={contrib_null_file} --override='TfModiscoWorkflow.min_metacluster_size = 1000' --contrib-wildcard={task}/profile/wn --only-task-regions --premade=modisco-50k {modisco_dir}/{task}/ --overwrite

### Visualizing the results

You can now use `MultipleModiscoResult` class to handle multiple `ModiscoResult` objects and visualize motifs discovered in all the tasks.

In [ ]:
# MultipleModiscoResult is a convenience wrapper around ModiscoResult
mf = ModiscoFileGroup({t: ModiscoFile(modisco_dir / t / 'modisco.h5') for t in tasks})

In [ ]:
# Plot all the patterns
for p in mf.patterns():
  p.plot("seq_ic", title=f"{p.name} ({mf.n_seqlets(p.name)})")
  plt.ylim([0, 2])
  sns.despine(top=True, bottom=True, right=True)

# Get motif instances with CWM scanning

To get motif instances in the genome, we will use CWM scanning using the `bpnet cwm-scan` command.

In [ ]:
!bpnet cwm-scan -h

We will run CWM scanning across the regions of all peaks, not just peaks of the TF for which TF-MoDISco was ran.

In [ ]:
# run CWM scanning for each task
for task in tasks:
  !bpnet cwm-scan {modisco_dir}/{task} {modisco_dir}/{task}/motif-instances.tsv.gz --contrib-file {contrib_file} --add-profile-features

In [ ]:
# Let's load the resulting table:
dfi = pd.read_csv(modisco_dir / 'Nanog/motif-instances.tsv.gz', sep='\t')

In [ ]:
dfi.head()

### Column description of motif instance table a.k.a. `dfi`

First seven columns are the most important ones. These represent the motif instance coordinates and the two scores derived from CWM scanning (contribution and match scores):
- chrom
- start
- end
- pattern name
- contribution score ([0, 1], 1 = highest contribution, 0=lowest contribution)
- strand. 
- CWM match score measuring the Jaccard similarity between CWM and the contribution scores ([0, 1], 1 = excellent match, 0=poor match)


Here is the full column name description:

***Motif instance position***
- `example_chrom` - chromosome name
- `pattern_start_abs` - chromosome start position (0-based, same as BED files)
- `pattern_end_abs` - chromosome end position (0-based, same as BED files)
- `pattern_len` - pattern length (same as `pattern_end_abs - pattern_start_abs`)
- `pattern` - pattern name from TF-MoDISco (e.g. `metacluster_0/pattern_1`)
- `pattern_short` - short pattern name from TF-MoDISco (e.g. `m0_p1`)
- `strand` - strand of the motif instance match
- `id` - unique ID for each motif instance (row number of the original pd.DataFrame)

***Scanned region information***
- `example_idx` - index of the scanned region. This corresponds to the item in the scanned `ContribFile`. 
- `example_start` - start position of the scanned region.
- `example_end` - end of the scanned region.
- `example_strand` - strand of the scanned region
- `example_interval_from_task` - `task` for which the scanned region was derived (e.g. Oct4 means that that region was an Oct4 peak specified in `dataspec.yml`).

***Relative motif region information***
- `pattern_start` - start position within the region
- `pattern_end` - end position within the regions
- `pattern_center` - center position within the regions

***PWM scan match***
- `seq_match` - classical PWM match score

***Contribution score amount***
- `contrib/{task}` - Absolute amount of the contribution score for task `{task}` at the motif instance position.
- `contrib_max` - maximum match value across all `contrib/{task}` columns
- `contrib_max_task` - task name for which `contrib/{task}` is maximal
- `contrib_weighted` - contribution score weighted across tasks. ***NOTE*** If only a single task was used to run modisco as it is the case in this tutorial, then `contrib/{task} = contrib_max = contrib_weighted`.
- `contrib_weighted_cat` - contribution score category (low, medium, high). These were determined by partitioning contrib_weighted_p according to the intervals (0, 0.33], (0.33, 0.66], (0.66, 1]

***CWM match***
- `match/{task}` - Jaccard similarity between the contribution score for task `{task}` and the CWM
- `match_max` - maximum match value across all `match/{task}` columns
- `match_max_task` - task name for which `match/{task}` is maximal
- `match_weighted` - contribution score weighted across tasks.  ***NOTE*** If only a single task was used to run modisco as it is the case in this tutorial, then `match/{task} = match_max = match_weighted`.
- `match_weighted_cat` - contribution score quantile

***Footprint evaluation (optional)***
- `{task}/profile_max` - maximal number of footprint count in the 70 bp window at the motif instance
- `{task}/profile_counts_max_ref` - number of counts at the position where the reference footprint is maximal (sum across strands).
- `{task}/profile_match` - symmetric KL divergence between the original footprint and the observed footprint (footprint = 70 bp wide)
- `{task}/profile_counts` - total number of counts of `{task}` in the 70 bp window at the motif instance


***NOTE***: All the columns ending with ***`_p`*** contain the CDF values (value between 0 and 1) using the original TF-MoDISco seqlets to determine the distribution. 

See the `bpnet.modisco.pattern_instances` module for more function to interact with that table. Throughout the whole codebase, that table is refered to as `dfi` (**d**ata-***f***rame of ***i***nstances).

## Locus visualization with motif instances higlighted

Let's visualize the same region as above, but this time highlighting differnet motif instances.

In [ ]:
from bpnet.utils import pd_col_prepend
from bpnet.modisco.pattern_instances import dfi2seqlets
from bpnet.modisco.utils import shorten_pattern, longer_pattern

In [ ]:
# First load all the motif instances simultaneously
dfi = pd.concat([pd.read_csv(modisco_dir / f'{task}/motif-instances.tsv.gz', sep='\t').
                 assign(tf=task).
                 pipe(pd_col_prepend, ['pattern', 'pattern_short'], prefix=task + "/")  # prefix the pattern names with task name
                 for task in tasks], sort=False)

In [ ]:
idx = 1  # we'll use the same idx as displayed before

# pattern instances in that locus
dfi[dfi.example_idx == idx]

# note that some instances have a poor match (match_weighted_p low)

In [ ]:
# convert dfi to Seqlet objects
seqlets = [s.shift(-xrange.start)
           for s in dfi2seqlets(dfi[dfi.example_idx == idx], short_name=True)]
seqlets

# # let's visualize these patterns
# mf = ModiscoFileGroup({t: ModiscoFile(modisco_dir / f'{t}/modisco.h5') for t in tasks})
# observed_patterns = {s.name for s in seqlets}  # get unique pattern names

# for pn in observed_patterns:
#   # NOTE: modisco requires the longer pattern version (e.g. {Task}/metacluster_0/pattern_1)
#   pn_long = longer_pattern(pn)
#   mf.get_pattern(pn_long).trim_seq_ic(0.08).plot('seq_ic')
#   sns.despine(top=True, bottom=True, right=True)

In [ ]:
# Visualize the locus with motif instances highlighted

# get the contribution scores and profile score for that example idx
xrange = slice(50, 150)
cf = ContribFile(contrib_file)
profiles = cf.get_profiles(idx=idx)
contrib_scores = cf.get_contrib(idx=idx)


# Let's focus only on the best match per track
dfi_best = dfi[dfi.example_idx == idx].sort_values("match_weighted_p", ascending=False).groupby('tf').first()

seqlets = [s.shift(-xrange.start)
           for s in dfi2seqlets(dfi_best, short_name=True)]


plot_tracks({**{'profile/' + k: to_neg(v[xrange]) for k,v in profiles.items()},
             **{'contrib/' + k:v[xrange] for k,v in contrib_scores.items()}},
           title=idx,
           rotate_y=0,
           fig_width=10,
           seqlets=[s.set_seqname('contrib/' + s.name.split("/")[0]) for s in seqlets], # plot seqlets to the 'contrib/Nanog' track
           fig_height_per_track=1);
sns.despine(top=True, right=True, bottom=True)

## ChIP-nexus heatmap visualization

In [ ]:
# get the chip-nexus data at the best motif isntances
dfi = pd.read_csv(modisco_dir / 'Oct4/motif-instances.tsv.gz', sep='\t')

cf = ContribFile(contrib_file)

# extract into `StackedSeqletContrib`
sc = cf.extract_dfi(dfi.
                    query("pattern_short == 'm0_p0'").
                    sort_values('contrib_weighted_p', ascending=False).iloc[:500], 
                    profile_width=50)

sc.plot(kind='profile_agg', figsize_tmpl=(3.3, 2));
sc.plot(kind='profile', figsize=(10, 8));
sc.plot(kind='seq',figsize_tmpl=(13, 10));
plt.title("Sequence");
# sc.plot(kind='contrib', figsize=(10, 8));

## De-novo sequence scanning 

Say we would like to determine motif instances in a new sequence. Here is how to do that.

In [ ]:
# Load the SeqModel
from bpnet.seqmodel import SeqModel

# In case you get `Can't get attribute '_make_skeleton_class'` error, please restart the runtime
# Runtime -> Reset runtimeb -> Yes
sm = SeqModel.from_mdir(model_dir)

In [ ]:
from bpnet.plot.tracks import filter_tracks, plot_tracks
from concise.preprocessing import encodeDNA
from collections import OrderedDict

# Core of the Oct4 enhancer sequence
seq = 'N'*23 + 'GGAGGAACTGGGTGTGGGGAGGTTGTAGCCCGACCCTGCCCCTCCCCCCAGGGAGGTTGAGAGTTCTGGGCAGACGGCAGATGCATAACAAAGGTGCATGATAGCTCTGCCCTGGGGGCAGAGAAGATGGTTGGGGAGGGGTCCCTCTCGTCCTA' + 'N'*22
seq_onehot = encodeDNA([seq]) # one-hot encode


# Compute contribution scores
contrib_scores = sm.contrib_score_all(seq_onehot, preact_only=True)
contrib_scores = [(f'Contrib {k}', seq_onehot[0] * v[0])
                  for k,v in contrib_scores.items()
                  if k.endswith("profile/wn")]  # keep only */profile/wn scores

# Make predictions
preds = sm.predict(seq_onehot)
preds = [(f"Pred {task}", to_neg(preds[f"{task}/profile"][0]) * np.exp(preds[f"{task}/counts"][0]))
         for task in sm.tasks]  # merge predictions

viz_dict = OrderedDict(preds + contrib_scores)

xlim = [50, 180]  # Focus only on the central
viz_dict = filter_tracks(viz_dict, xlim)

In [ ]:
# determine the location of the first instance
from bpnet.modisco.files import ModiscoFile

mf = ModiscoFile(modisco_dir / 'Oct4/modisco.h5')
pattern = mf.get_pattern("metacluster_0/pattern_0").trim_seq_ic(0.08)  # get trimmed pattern
task = 'Oct4'

# scan the contribution scores + DNA sequence
match, contribution = pattern.scan_contribution({task: dict(contrib_scores)[f'Contrib {task}/profile/wn'][np.newaxis]}, 
                                                 hyp_contrib=None, tasks=[task], n_jobs=1, verbose=False)
seq_match = pattern.scan_seq(seq_onehot, n_jobs=1, verbose=False)

# load the normalization table
dfi_norm = pd.read_csv(f"{modisco_dir}/{task}/cwm-scan-seqlets.trim-frac=0.08.csv.gz")

# get the motif instance table
dfm = pattern.get_instances([task], match, contribution, seq_match,
                            norm_df=dfi_norm[dfi_norm.pattern == pattern.name],
                            verbose=False, plot=False)
dfm

In [ ]:
# convert dfi to Seqlet objects and plot it
from bpnet.modisco.pattern_instances import dfi2seqlets

seqlets = [s.shift(-xlim[0]).set_seqname(f'Contrib {task}/profile/wn')
         for s in dfi2seqlets(dfm, short_name=True)]

plot_tracks(viz_dict,
            fig_height_per_track=1,
            fig_width=10,
            seqlets=seqlets,
            rotate_y=0);
sns.despine(top=True, right=True, bottom=True)

## Motif spacing visualization

Here, we will visualize the motif spacing between the two Nanog motifs

In [ ]:
from bpnet.modisco.files import ModiscoFile

# Let's load the resulting table:
dfi = pd.read_csv(modisco_dir / 'Nanog/motif-instances.tsv.gz', sep='\t')
# visualize spacing for Nanog/metacluster_0/pattern_0
mf = ModiscoFile(modisco_dir / 'Nanog/modisco.h5')
p = mf.get_pattern('metacluster_0/pattern_0')
p.plot(['seq_ic', "contrib"], rotate_y=0);

In [ ]:
# create a table of motif_paris
from bpnet.modisco.pattern_instances import motif_pair_dfi  # main function for motif spacing


dfi_subset = dfi[dfi.pattern == p.name]  # use only the first pattern
dfi_subset['pattern_name'] = 'Nanog'  # motif_pair_dfi requires `pattern_name` column

dfab = motif_pair_dfi(dfi_subset, ['Nanog', 'Nanog'])
print(len(dfab))
dfab.head()

In [ ]:
plt.hist(dfab[dfab.strand_combination == "++"].center_diff, bins=np.arange(40)+0.5)
plt.xlabel("Motif distance")
plt.ylabel("Frequency");

## FAQ

### How do I score new regions not present in  the contribution file?


If you would like to get motif instances for regions not present in the contribution file, the easiest is to first generate the contribution file for your regions of interest using `bpnet contrib ... --regions=regions.bed`  and then run `cwm-scan` with `--contrib-file` pointing to this new contribution file.

# Simulate motif spacing


Showcase the `BPNet` model API
    - visualize the locus with contrib scores + predictions
    - motif spacing + simulation

In [ ]:
# Simulate Oct4-Sox2 and Nanog spacing
from bpnet.BPNet import BPNetSeqModel

bpn = BPNetSeqModel(sm)  # wrap SeqModel to BPNetSeqModel to get `sim_pred` method


plot_tracks(bpn.sim_pred(central_motif='TTTGCATAACAA', side_motif='AGCCATCA', side_distances=[115]), 
            fig_height_per_track=1,
            fig_width=10,
            rotate_y=0,
            title="distance = 15");
sns.despine(top=True, right=True, bottom=True)


plot_tracks(bpn.sim_pred(central_motif='TTTGCATAACAA', side_motif='AGCCATCA', side_distances=[150]), 
            fig_height_per_track=1,
            fig_width=10,
            rotate_y=0,
            title='distance = 50');
sns.despine(top=True, right=True, bottom=True)

In [ ]:
# run simulation for the whole range
# quantify the profile height (profile/* features)
from bpnet.simulate import generate_sim

res = generate_sim(bpn, central_motif='AGCCATCA', side_motif='TTTGCATAACAA', side_distances=np.arange(110, 170), center_coords=[65, 135], contribution=[], correct=True)

dfs, profiles = res
dfs.head()

In [ ]:
from plotnine import *
import plotnine

plotnine.options.figure_size = (4, 2)
(ggplot(aes(x='distance', y='profile/counts_max_ref_frac'), dfs[dfs.task=='Nanog']) + 
 geom_line() + 
 geom_hline(yintercept=1, alpha=0.5) + 
 ylim([0, 2]) +
 theme_classic())

# FAQ

### I want to train a model on ChIP-seq. How can I do this?

Follow the same procedure as for ChIP-nexus.

### I want to train a model on DNase-seq or ATAC-seq. How can I do this?

The key difference between DNase-seq and ChIP-seq/exo is that DNase-seq coverage is not strand specific. Hence a single BigWig file is required. By contrast, ChIP-seq required two BigWigs - one for the positive and one for the negative strand. Beware that controlling for DNase biases is still an open question and you should think carefully about it.

Otherwise, you can just specify a similar `dataspec.yml` as before:

# Notes / Ideas

- Allow the user to download the output files instead of running the model by adding an if else in each chapter
  - e.g. make each chapter independent of each other
  - required files:
    - contrib: `seq_model.pkl`, `config.gin.json`, `dataspec.yml`
    - modisco-run: `contrib_file`, `null_contrib_file`
    - cwm-scan: `modisco.h5`, `modisco-run.subset-contrib-file.npy`, `modisco-run.kwargs.json`
    - reports: ...?
- bpnet 
- [ ] go through all the paper notebooks and list useful things to show
  - how do I visualize a particular locus with all the motif instances + contrib scores?
- [ ] shall we rename `example->regions` in dfi?

In [ ]:
# Export the generated files

# from google.colab import drive
# drive.mount('/gdrive')

In [ ]:
# bpnet_demo_dir = '/gdrive/My\ Drive/projects/chipnexus/data/bpnet-demo'
# !mkdir -p {bpnet_demo_dir}
# !cp -R {model_dir} {bpnet_demo_dir}/